### Importación de librerías

In [ ]:
import cv2
import os
import mediapipe as mp

### Definición de variables

In [ ]:
DATASET_NAME = "../../dataset/gestos_cuerpo"
LABELS = {
    0: "0_Neutro",
    1: "1_Mano_Der_Arriba",      # Brazo levantado
    2: "2_Mano_Izq_Arriba",      # Brazo levantado
    3: "3_Punos_Cerrados",       # Ambas manos
    4: "4_Pulgar_Arriba",        # Gesto simple
    5: "5_Victoria",             # Gesto victoria
    6: "6_Rock",                 # Gesto rock
    7: "7_Llamada",              # Gesto llamada
    8: "8_Ok"                    # Gesto ok
}

### Lógica de guardado de imágenes

In [ ]:
# Crear carpetas
if not os.path.exists(DATASET_NAME):
    os.makedirs(DATASET_NAME)

for key, nombre_carpeta in LABELS.items():
    path = os.path.join(DATASET_NAME, nombre_carpeta)
    if not os.path.exists(path):
        os.makedirs(path)

# === MEDIAPIPE HOLISTIC (MEJOR PARA BRAZOS + MANOS) ===
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Usamos Holistic para ver hombros y codos también
holistic = mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)

# Contadores
contadores = {}
for key in LABELS:
    path = os.path.join(DATASET_NAME, LABELS[key])
    contadores[key] = len(os.listdir(path))

print(f"--- RECOLECTOR DE CUERPO Y MANOS ---")
print(f"Guardando en: {DATASET_NAME}")

while True:
    ret, frame = cap.read()
    if not ret: break
    
    # Espejo
    frame = cv2.flip(frame, 1)  
    frame_display = frame.copy()
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(frame_rgb)
    
    cuerpo_detectado = False
    
    # Dibujar Pose (Brazos/Cuerpo)
    if results.pose_landmarks:
        cuerpo_detectado = True
        mp_drawing.draw_landmarks(
            frame_display,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    # Dibujar Manos (Izquierda y Derecha)
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(frame_display, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(frame_display, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    # === INTERFAZ ===
    fuente = cv2.FONT_HERSHEY_SIMPLEX
    
    # Panel superior (Resumen)
    info_counts = f"N:{contadores[0]} | MD:{contadores[1]} | MI:{contadores[2]} | P:{contadores[3]} | V:{contadores[5]} | Rock:{contadores[6]} | Ll:{contadores[7]} | Ok:{contadores[8]}"
    cv2.putText(frame_display, info_counts, (10, 30), fuente, 0.5, (0, 255, 255), 1)

    # Panel inferior (Instrucciones)
    lineas = [
        "0:Neutro | 1:Mano Der Up | 2:Mano Izq Up",
        "3:Punos | 4:Pulgar | 5:Victoria",
        "6:Rock | 7:Llamada | 8:Ok | q:Salir"
    ]
    
    y0 = 400
    for i, linea in enumerate(lineas):
        y = y0 + i * 25
        # Fondo negro para leer mejor
        (w, h), _ = cv2.getTextSize(linea, fuente, 0.6, 1)
        cv2.rectangle(frame_display, (10, y - h - 5), (10 + w + 10, y + 5), (0,0,0), -1)
        cv2.putText(frame_display, linea, (15, y), fuente, 0.6, (0, 255, 255), 1)

    if not cuerpo_detectado:
        cv2.putText(frame_display, "NO SE DETECTA CUERPO", (10, 200), fuente, 0.8, (0, 0, 255), 2)

    cv2.imshow('Recolector Cuerpo/Manos', frame_display)
    
    # === GUARDADO ===
    k = cv2.waitKey(1)
    if k == ord('q'): break
    
    clase = -1
    # Mapeo de teclas numéricas
    if 48 <= k <= 56: # ASCII del 0 al 8
        clase = k - 48
    
    if clase != -1:
        carpeta = LABELS[clase]
        idx = contadores[clase]
        path_img = os.path.join(DATASET_NAME, carpeta, f"{carpeta}_{idx}.jpg")
        cv2.imwrite(path_img, frame) # Guardamos frame limpio
        print(f"Guardado: {path_img}")
        contadores[clase] += 1

cap.release()
cv2.destroyAllWindows()